In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkFiles
from pyspark.sql import DataFrame
import pyspark.sql.functions as F

In [2]:
# get or create Spark session

app_name = "amazon-giftcard"
spark = SparkSession.builder.appName(app_name).getOrCreate()

In [3]:
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Gift_Card_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.csv("file://"+SparkFiles.get("amazon_reviews_us_Gift_Card_v1_00.tsv.gz"), header=True,sep='\t', inferSchema= True)

In [4]:
df.show(3)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
marketplace|customer_id| review_id|product_id|product_parent| product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase| review_headline| review_body| review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
 US| 24371595|R27ZP1F1CD0C3Y|B004LLIL5A| 346014806|Amazon eGift Card...| Gift Card| 5| 0| 0| N| Y| Five Stars|Great birthday gi...|2015-08-31 00:00:00|
 US| 42489718| RJ7RSBCHUDNNE|B004LLIKVU| 473048287|Amazon.com eGift ...| Gift Card| 5| 0| 0| N| Y|Gift card for the...|It's an Amazon gi...|2015-08-31 00:00:00|
 US| 861463|R1HVYBSKLQJI5S|B00IX1I3G6| 926539283|Amazon.com Gift C...| Gift Card| 5| 0| 0| N| Y| Five Stars| Good|2015-08-31 00:00:00|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
only showing top 3 rows

In [5]:
df.printSchema()

root
-- marketplace: string (nullable = true)
-- customer_id: integer (nullable = true)
-- review_id: string (nullable = true)
-- product_id: string (nullable = true)
-- product_parent: integer (nullable = true)
-- product_title: string (nullable = true)
-- product_category: string (nullable = true)
-- star_rating: integer (nullable = true)
-- helpful_votes: integer (nullable = true)
-- total_votes: integer (nullable = true)
-- vine: string (nullable = true)
-- verified_purchase: string (nullable = true)
-- review_headline: string (nullable = true)
-- review_body: string (nullable = true)
-- review_date: timestamp (nullable = true)

In [6]:
df.count()

Out[62]: 149086

In [7]:
customers=df.groupBy(df.customer_id).count()
customers_df=customers.selectExpr('customer_id','count as customer_count')

In [8]:
review_id_table_df=df.selectExpr("review_id", "customer_id","product_id","product_parent","review_date")
products_df=df.selectExpr("product_id", "product_title")
vine_table_df=df.selectExpr("review_id", "star_rating","helpful_votes","total_votes","vine")


In [9]:
products_df.head()

Out[56]: Row(product_id='B004LLIL5A', product_title='Amazon eGift Card - Celebrate')

In [10]:
import json
def get_connection(secret_value):
  return json.loads(secret_value['SecretString'])

In [11]:
connection = get_connection(secret_value)

# Postgres credentials
jdbcHostname = connection['host']
jdbcPort = connection['port']
jdbcDatabase = "postgres"
dialect = "postgresql"
jdbcUsername = connection['username']
jdbcPassword = connection['password']

jdbcUrl = f"jdbc:{dialect}://{jdbcHostname}:{jdbcPort}/{jdbcDatabase}"
connectionProperties = {
  "user" : jdbcUsername,
  "password" : jdbcPassword,
  "driver" : "org.postgresql.Driver" 
}

In [12]:
table = "review_id_table"
mode = "overwrite" # options are: error, append, overwrite
review_id_table_df.write.jdbc(jdbcUrl, table, mode, connectionProperties)

In [13]:
table = "products"
mode = "overwrite" # options are: error, append, overwrite
products_df.write.jdbc(jdbcUrl, table, mode, connectionProperties)

In [14]:
table = "customers"
mode = "overwrite" # options are: error, append, overwrite
customers_df.write.jdbc(jdbcUrl, table, mode, connectionProperties)

In [15]:
table = "vine_table"
mode = "overwrite" # options are: error, append, overwrite
vine_table_df.write.jdbc(jdbcUrl, table, mode, connectionProperties)